In [ ]:
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from collections import defaultdict
import re
from nltk.stem.porter import PorterStemmer
import heapq
import json
import gc
porter = PorterStemmer()

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_symbols(line):
    return re.sub('[^A-Za-z0-9\s]+', '', line).lower()

"""
we are using this to change list into set while dumping json into file
"""
class SetEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, set):
      return list(obj)
    return json.JSONEncoder.default(self, obj)

In [ ]:
import csv
import sys
csv.field_size_limit(sys.maxsize) 

131072

In [ ]:
# going to our folder
% cd /content/drive/MyDrive/
# % rm -rf OP*

/content/drive/MyDrive


In [ ]:
BLOCK_SIZE = 100000 

In [ ]:
def bsbi():
  freq_dict = defaultdict(set)
  with open('gutenberg_data.csv') as f:
    next(f) 
    csv_file = csv.reader(f)
    total_files = 0
    i = 0
    current_block = 0
    for line in csv_file:
      title, author, link, id, bookshelf, text = line
      i += 1
      
      for word in text.split():
        word = remove_symbols(word)
        if word and word not in stop_words:
          word = porter.stem(word)
          if word not in freq_dict:
      
            current_block += 1
          if not freq_dict[word].__contains__(id):
            freq_dict[word].add(id)
            current_block += 1
        if current_block >= BLOCK_SIZE:
      
          sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) 
          with open(f'./ss/OP{total_files}.txt', 'w') as  f:
      
            for word_id, docs in sorted_list:
              f.write(word_id)
              for doc_id in docs:
                f.write(f' {doc_id}')
              f.write('\n')
          current_block = 0
          freq_dict.clear()
          total_files += 1
          print(i, ' rows done')
      
    sorted_list = sorted(freq_dict.items(), key=lambda _: _[0]) 
    if len(sorted_list) > 0:
      with open(f'./ss/OP{total_files}.txt', 'w') as  f:
    
        for word_id, docs in sorted_list:
          f.write(word_id)
          for doc_id in docs:
            f.write(f' {doc_id}')
          f.write('\n')
      current_block = 0
      freq_dict.clear()
      total_files += 1
bsbi()

15  rows done
23  rows done
41  rows done
54  rows done
67  rows done
81  rows done
96  rows done
116  rows done
130  rows done
146  rows done
157  rows done
181  rows done
207  rows done
220  rows done
254  rows done
271  rows done
280  rows done
291  rows done
302  rows done
332  rows done
353  rows done
371  rows done
387  rows done
400  rows done
444  rows done
456  rows done
468  rows done
476  rows done
488  rows done
499  rows done
506  rows done
509  rows done
518  rows done
526  rows done
533  rows done
541  rows done
548  rows done
565  rows done
590  rows done
606  rows done
616  rows done
627  rows done
634  rows done
641  rows done
649  rows done
660  rows done
678  rows done
685  rows done
693  rows done
703  rows done
718  rows done
732  rows done
740  rows done
749  rows done
757  rows done
762  rows done
772  rows done
779  rows done
788  rows done
796  rows done
804  rows done
817  rows done
828  rows done
839  rows done
848  rows done
857  rows done
866  rows done
87

In [ ]:
file_names = [f'./ss/OP{i}.txt' for i in range(925)]
file_pointers = [open(i) for i in file_names]

FileNotFoundError: ignored

In [ ]:
"""
This algo is called external merge
In python I took help of heaps to do the merging
A great answer of what I did is at this link,
https://stackoverflow.com/questions/1001569/python-class-to-merge-sorted-files-how-can-this-be-improved
^Make sure to take a look at this to see what we are really doing

"""

# here we are using yeild so we are JUST READING ONE LINE at a time
# you can read more about it by searching generators in python
def decorated_file(f, key):
  for line in f:
    yield (key(line), line)

files = map(open, file_names)
outfile = open('./ss/merged.txt', 'w')

def key_fn(line):
    return line.split(' ', 2)[0] # returning word_id

"""
The simple sorting version would be like below

for line in heapq.merge(*[decorated_file(f, keyfunc) for f in files]):
    outfile.write(line[1])

so if there'd be 2 files 
a 1 2 3
b 3 4 5

and 

a 6 7 8
b 8 9 10

o/p would be
a 1 2 3
a 6 7 8
b 3 4 5
b 8 9 10
So I kind of did lil bit of modification to do what we want in one pass
TIME: 6m 14s (for ~1k files of size ~620kb each)

a 1 2 3 6 7 8
b 3 4 5 8 9 10
"""
prev = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
  # to understand this, you can do this over small number of sorted files and try to print line
  if prev != line[0]:
    # if we have new word, make sure to add new line at first
    outfile.write(f'\n{line[1].strip()}')
    prev = line[0]
  # if we have same word yet, put a space and add other ids
  else:
    # line[1][len(line[0]):] => We are removing the word_id string and then writing the line
    outfile.write(f' {line[1][len(line[0]):].strip()}')
for i in file_pointers:
  i.close()
outfile.close()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive")
!ls

 2_PSR_CS609.gdoc	   gutenberg_data.csv	 tA-converted.docx
 Ajay.mp4		   Network_Devices.ppt	'TAship Form- SEPT.gdoc'
 Classroom		  'Nwt Video'		'TAship Form- SEPT.pdf'
'Colab Notebooks'	   opt
'gutenberg_data (1).csv'   ss


In [ ]:
file_names = [f'./ss/OP{i}.txt' for i in range(925)]
file_pointers = [open(i) for i in file_names]

In [ ]:
# Using External Merge algorithm
# Credits: https://stackoverflow.com/questions/1001569/python-class-to-merge-sorted-files-how-can-this-be-improved

def decorated_file(f, key):
  for line in f:
    yield (key(line), line)

files = map(open, file_names)
outfile = open('./ss/merged.txt', 'w')

def key_fn(line):
    return line.split(' ', 2)[0] # returning word_id

prev = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
  # to understand this, you can do this over small number of sorted files and try to print line
  if prev != line[0]:
    # if we have new word, make sure to add new line at first
    outfile.write(f'\n{line[1].strip()}')
    prev = line[0]
  # if we have same word yet, put a space and add other ids
  else:
    # line[1][len(line[0]):] => We are removing the word_id string and then writing the line
    outfile.write(f' {line[1][len(line[0]):].strip()}')
for i in file_pointers:
  i.close()
outfile.close()

In [ ]:
for i in file_pointers:
  i.close()
outfile.close()